<a href="https://colab.research.google.com/github/sofiadellarosa/FinalProjectDS/blob/main/DSproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#import github stuff
%cd /content
!git clone https://github.com/sofiadellarosa/FinalProjectDS.git
%cd /content/drive/MyDrive/project_files


import pandas as pd
import numpy as np
import re
pd.set_option("display.max_rows", None, "display.max_columns", None)

/content
fatal: destination path 'FinalProjectDS' already exists and is not an empty directory.
[Errno 2] No such file or directory: '/content/drive/MyDrive/project_files'
/content


In [ ]:
disaster_declaration = pd.read_csv('/content/drive/MyDrive/Datasets/DisasterDeclarationsSummaries.csv') # FEMA’s Disaster Declarations Summaries
hpi = pd.read_csv('/content/drive/MyDrive/Datasets/hpi_at_bdl_zip5.csv') #Federal Housing Finance Agency's Annual House Price Indexes

<ipython-input-3-084b96db48a2>:1: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  disaster_declaration = pd.read_csv('/content/drive/MyDrive/Datasets/DisasterDeclarationsSummaries.csv') # FEMA’s Disaster Declarations Summaries
<ipython-input-3-084b96db48a2>:2: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  hpi = pd.read_csv('/content/drive/MyDrive/Datasets/hpi_at_bdl_zip5.csv') #Federal Housing Finance Agency's Annual House Price Indexes


In [ ]:
disaster_declaration.head()

,femaDeclarationString,disasterNumber,state,declarationType,declarationDate,fyDeclared,incidentType,declarationTitle,ihProgramDeclared,iaProgramDeclared,paProgramDeclared,hmProgramDeclared,incidentBeginDate,incidentEndDate,disasterCloseoutDate,tribalRequest,fipsStateCode,fipsCountyCode,placeCode,designatedArea,declarationRequestNumber,lastIAFilingDate,incidentId,region,designatedIncidentTypes,lastRefresh,hash,id
0,FM-5529-OR,5529,OR,FM,2024-08-09T00:00:00.000Z,2024,Fire,LEE FALLS FIRE,0,0,1,1,2024-08-08T00:00:00.000Z,NaN,NaN,0,41,67,99067,Washington (County),24122,NaN,2024081001,10,R,2024-08-27T18:22:14.800Z,ae87cf3c6ed795015b714af7166c7c295b2b67c7,09e3f81a-5e16-4b72-b317-1c64e0cfa59c
1,FM-5528-OR,5528,OR,FM,2024-08-06T00:00:00.000Z,2024,Fire,ELK LANE FIRE,0,0,1,1,2024-08-04T00:00:00.000Z,NaN,NaN,0,41,31,99031,Jefferson (County),24116,NaN,2024080701,10,R,2024-08-27T18:22:14.800Z,432cf0995c47e3895cea696ede5621b810460501,59983f89-30bf-4888-b21b-62e8d57d9aac
2,FM-5527-OR,5527,OR,FM,2024-08-02T00:00:00.000Z,2024,Fire,MILE MARKER 132 FIRE,0,0,1,1,2024-08-02T00:00:00.000Z,NaN,NaN,0,41,17,99017,Deschutes (County),24111,NaN,2024080301,10,R,2024-08-27T18:22:14.800Z,2f21d90cb6bc64b0d4121aa3f18d852bbb4b11fa,8d13ecf0-bc2f-496b-8c9f-b2e73da832a0
3,FM-5521-WA,5521,WA,FM,2024-07-26T00:00:00.000Z,2024,Fire,RETREAT FIRE,0,0,1,1,2024-07-23T00:00:00.000Z,NaN,NaN,0,53,77,99077,Yakima (County),24098,NaN,2024072601,10,R,2024-08-27T18:22:14.800Z,e205f5a4b56cadd811661c07fc8a8143b6c8c604,bc02a243-2458-4af3-84e3-b9928bd1d41d
4,FM-5520-ID,5520,ID,FM,2024-07-26T00:00:00.000Z,2024,Fire,GWEN FIRE,0,0,1,1,2024-07-25T00:00:00.000Z,NaN,NaN,0,16,0,57340,Nez Perce Indian Reservation,24099,NaN,2024072602,10,R,2024-08-27T18:22:14.800Z,ba084caa4ee18c9c0c4383346e7669d4af4ab9b7,85cccdec-5e22-436f-a3e7-acd7340172cf


In [ ]:
hpi.head()

,HPI for Five-Digit ZIP Codes (All-Transactions Index)\nExperimental Indexes Showing Cumulative (Nominal) Annual Appreciation,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,* These annual ZIP5 indexes should be consider...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"** For tracking and feedback purposes, please ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Last updated: March 26, 2024.",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Not Seasonally Adjusted (NSA),NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# np="/content/drive/MyDrive/project_files/DSproject.ipynb"
# print(np)

%%shell
jupyter nbconvert --to html "/content/drive/MyDrive/project_files/DSproject.ipynb"

Traceback (most recent call last):
  File "/usr/local/bin/jupyter-nbconvert", line 4, in <module>
    from nbconvert.nbconvertapp import main
  File "/usr/local/lib/python3.11/dist-packages/nbconvert/__init__.py", line 6, in <module>
    from . import filters, postprocessors, preprocessors, writers
  File "/usr/local/lib/python3.11/dist-packages/nbconvert/filters/__init__.py", line 8, in <module>
    from .markdown import (
  File "/usr/local/lib/python3.11/dist-packages/nbconvert/filters/markdown.py", line 16, in <module>
    from .markdown_mistune import markdown2html_mistune
  File "/usr/local/lib/python3.11/dist-packages/nbconvert/filters/markdown_mistune.py", line 21, in <module>
    from nbconvert.filters.strings import add_anchor
  File "/usr/local/lib/python3.11/dist-packages/nbconvert/filters/strings.py", line 23, in <module>
    from nbconvert.preprocessors.sanitize import _get_default_css_sanitizer
object address  : 0x79a23491c280
object refcount : 2
object type     : 0x9d5e

CalledProcessError: Command 'jupyter nbconvert --to html "/content/drive/MyDrive/project_files/DSproject.ipynb"
' died with <Signals.SIGINT: 2>.

Project Name: Housing Market Change after Natural Disaster

Team Members: Brandon Butler and Sofia Della Rosa

Project Description:
For this project we will analyze how certain natural disasters in different regions impact the housing market in the United States. We can use that information to see if property values decline after disasters and how quickly they recover. The natural disasters we will be looking at are: hurricanes, earthquake, drought, fire, flood, and tornadoes.

Collaboration Plan:
For this project, we have a collaboration plan to make sure we have efficient workflow and collaboration. We meet in person every Wednesday to work on the project together. We communicate through text messages. We also use a shared Google Calendar to set up meetings and keep track of deadlines. We also set up a shared Google Drive where we store project documents and files. Inside the Google Drive is a To-Do Note file that we use to track our project tasks. Lastly, we made a GitHub repository to manage and coordinate our code efficiently.

Table 1:
The first dataset we are working with is the Federal Housing Finance Agency's Annual House Price Indexes. This dataset contains House Price Index (HPI) data at the state and zip code levels. We chose this dataset because it provides a reliable measure of housing market trends over time. This will allow us to analyze how property values change in response to natural disasters and then track their recovery rates. A question we have for this dataset is how do house prices change in the years following a natural disaster? Another question is how long does it take for them to return to pre-disaster levels?

Small amount of ETL including 3-5 interesting stats
One graph

Table 2:
The next dataset we are working with is FEMA’s Disaster Declarations Summaries. This dataset lists all federally declared disasters with relevant attributes such as date, type, and duration. We chose this dataset because it provides detailed records of natural disasters. This will allow us to identify when and where disasters occurred and then analyze their impact on housing markets in affected regions. A question we have for this dataset is what types of natural disasters have the most significant and longest impact on housing prices in affected areas?

Small amount of ETL including 3-5 interesting stats
One Graph


Project Goals:

